In [2]:
export ECMWF_API_URL="https://api.ecmwf.int/v1"
export ECMWF_API_KEY="35fba4b59a1581554f602f6541b09654"
export ECMWF_API_EMAIL="minhyuk.nam@outlook.com"

SyntaxError: invalid syntax (<ipython-input-2-ca0295b8eca9>, line 1)

In [14]:
server = ECMWFDataServer(url="https://api.ecmwf.int/v1",key="35fba4b59a1581554f602f6541b09654",email="minhyuk.nam@outlook.com")

server.retrieve({
    "class": "mc",
    "dataset": "cams_nrealtime",
    "date": "2019-01-01/to/2020-07-31",
    "expver": "0001",
    "levtype": "sfc",
    "param": "72.210/73.210/125.210/126.210/127.210",
    "step": "3/6/9/12",
    "stream": "oper",
    "time": "00:00:00/12:00:00",
    "type": "fc",
    "target": "output"
})

2020-09-18 09:59:35 ECMWF API python library 1.5.6
2020-09-18 09:59:35 ECMWF API at https://api.ecmwf.int/v1
2020-09-18 09:59:37 Welcome Minhyuk Nam
2020-09-18 09:59:39 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2020-09-18 09:59:40 Request submitted
2020-09-18 09:59:40 Request id: 5f64067cd8c0fd70e7b4f62c
2020-09-18 09:59:40 Request is submitted
2020-09-18 09:59:43 Request is active
Error contacting the WebAPI, retrying in 60 seconds ...


KeyboardInterrupt: 

In [34]:
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import datetime
import os

In [35]:
directory = "/Users/minhyuk/Dropbox/RA/COVID19/weather-conditions-COVID19/WeatherDataCollection/templates"
os.chdir(directory)
#os.getcwd()
#os.system("dir")

## Set Template and Files

Data for gases (ozone, sulfur dioxide, and nitrogen dioxide) and data for particulate matter are stored in a different batch of files. Further, different templates are used depending on the dataset to be gathered. Change following boolean to decide which files to read.

In [36]:
read_pm_files = False

In [37]:
template_file_name = "US_pollution_template.xlsx"
#template_file_name = "global_simple_pollution_template.xlsx"
#template_file_name = "city granular_template.xlsx"
#template_file_name = "1072_pollution_template.xlsx"

In [38]:
template_file_name[:-14]

'US_pollution'

In [39]:
if read_pm_files:
    pm_df = pd.read_excel(template_file_name)
else:
    no2_df = pd.read_excel(template_file_name)
    o3_df = pd.read_excel(template_file_name)
    so2_df = pd.read_excel(template_file_name)

In [40]:
if read_pm_files:
    lookup_lats, lookup_longs = pm_df["Lat"], pm_df["Long"]
else:
#    lookup_lats, lookup_longs = pm_df["Lat"], pm_df["Long"]
    lookup_lats, lookup_longs = no2_df["Lat"], no2_df["Long"]

The following returns the lat/long indices closest to a desired point, and is used for reading from the netCDF4 layer.

In [41]:
def getClosest_ij(lats, longs, latpt, longpt):
    #Manhattan Distance
    i = abs(lats - latpt).argmin()
    j = abs(longs - longpt).argmin()
    return i,j

In [42]:
def fill_dataframes(month, month_length, year):
    for i in range(month_length):
        no2_col = []
        o3_col = []
        so2_col = []
        pm_col = []
        for lat, long in zip(lookup_lats, lookup_longs):
            iy, ix = getClosest_ij(lats, lons, lat, long)
            if read_pm_files:
                pm_col.append(pm[i][iy][ix])
            else:
                so2_col.append(so2[i][iy][ix])
                no2_col.append(no2[i][iy][ix])
                o3_col.append(o3[i][iy][ix])
            
        key = datetime.datetime(year, month, i+1).strftime("%Y-%m-%d")
        print(key)
        
        if read_pm_files:
            pm_df[key] = pm_col
        else:
            so2_df[key] = so2_col
            no2_df[key] = no2_col
            o3_df[key] = o3_col

The following are maps from file names to month, number of days, and year triples, with the triples being used to group the data by data. Notice that for the gases argument map (ozone, sulfur dioxide, nitrogen dioxide), July 2017 is commented out - this month is handled separately due to a shift in ECMWF's reporting structure.

In [43]:
pm_argument_map = {
    "1Jan2019":(1,31,2019),
    "2Feb2019":(2,28,2019),
    "3March2019":(3,31,2019),
    "4April2019":(4,30,2019),
    "5May2019":(5,31,2019),
    "6June2019":(6,30,2019),
    "7July2019":(7,31,2019),
    "8Auguest2019":(8,31,2019),
    "9September2019":(9,30,2019),
    "10October2019":(10,31,2019),
    "11November2019":(11,30,2019),
    "Dec2019":(12,31,2019),
    "Jan2020":(1,31,2020),
    "Feb2020":(2,29,2020),
    "March2020":(3,31,2020),
    "April2020":(4,30,2020)
}

In [44]:
gases_argument_map = {
    "1 Jan2019":(1,31,2019),
    "2 Feb2019":(2,28,2019),
    "3 March2019":(3,31,2019),
    "4 April2019":(4,30,2019),
    "5 May2019":(5,31,2019),
    "6 June2019":(6,30,2019),
    #"7 July2019":(7,31,2019),
    "8 Aug2019":(8,31,2019),
    "9 Sep2019":(9,30,2019),
    "10 Oct2019":(10,31,2019),
    "11 Nov2019":(11,30,2019),
    "12 Dec2019":(12,31,2019),
    "Jan2020":(1,31,2020),
    "Feb2020":(2,29,2020),
    "March2020":(3,31,2020),
    "April2020":(4,30,2020)
}

In [45]:
new_argument_map = {
    "April2020":(4,30,2020)    
#    "May2020":(5,31,2020),
#    "June2020":(6,30,2020),
#    "July2020":(7,31,2020),
#    "August2020":(8,31,2020)
}

#f = Dataset("April2020" + "_level0.nc", format="NETCDF4")
#f.variables['no'][1][:]


In [46]:
if read_pm_files:
#    month_argument_map = pm_argument_map
    month_argument_map = new_argument_map

else:
#    month_argument_map = gases_argument_map
    month_argument_map = new_argument_map
    
for file in month_argument_map:
    f = Dataset(file + "_level60.nc", format="NETCDF4")

    lats, lons = f.variables["latitude"][:], f.variables["longitude"][:]
    if read_pm_files:
        pm = f.variables['pm2p5'][:]
    else:
        no2 = f.variables['no2'][:]
        o3 = f.variables['go3'][:]
        so2 = f.variables['so2'][:]

    arg = month_argument_map[file]
    fill_dataframes(arg[0], arg[1], arg[2])

2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30


In [47]:
f.variables.keys()
#for file in month_argument_map:
#    arg = month_argument_map[file]
#    print(arg)
#f.variables['time']
#f.variables['pm2p5'][:]
#so2_df

odict_keys(['longitude', 'latitude', 'time', 'aermr01', 'aermr02', 'aermr03', 'aermr04', 'ch4_c', 'aermr05', 'aermr06', 'hno3', 'aermr07', 'aermr08', 'aermr09', 'aermr10', 'aermr11', 'pan', 'c5h8', 'no', 'oh', 'c2h6', 'c3h8', 'no2', 'so2', 'co', 'hcho', 't', 'q', 'vo', 'd', 'go3', 'aermr16', 'aermr17', 'aermr18'])

In [48]:
if read_pm_files:
    doc_name = template_file_name[:-14]+"_pm"
else:
    doc_name = template_file_name[:-14]+"_gases"

In [49]:
writer = pd.ExcelWriter(doc_name+'.xlsx', engine='xlsxwriter')

In [50]:
if read_pm_files:
    pm_df.to_excel(writer, sheet_name = "pm2p5", index = False)
else:
    o3_df.to_excel(writer, sheet_name = "o3", index = False)
    no2_df.to_excel(writer, sheet_name = "no2", index = False)
    so2_df.to_excel(writer, sheet_name = "so2", index = False)

In [51]:
writer.close()


## Manually Reading July2019 Gas Data

ECMWF shifted their reporting for ozone, sulfur dioxide, and nitrogen dioxide from 60 levels of vertical resolution to 137 (see https://confluence.ecmwf.int/display/COPSRV/Implementation+of+IFS+cycle+46r1), so we read both and patch them together at their transition date, which was July 9 2019.

In [ ]:
data60 = Dataset("7 July2019 level 60 up to here.nc", format="NETCDF4")

In [ ]:
data137 = Dataset("7 Jul2019 level 137 from here.nc", format="NETCDF4")

In [ ]:
template_file_name = "US_pollution_template.xlsx"
#template_file_name = "global_simple_pollution_template.xlsx"
#template_file_name = "city granular_template.xlsx"
#template_file_name = "1072_pollution_template.xlsx"

In [ ]:
no2_df = pd.read_excel(template_file_name)
o3_df = pd.read_excel(template_file_name)
so2_df = pd.read_excel(template_file_name)

In [ ]:
lookup_lats, lookup_longs = no2_df["Lat"], no2_df["Long"]

In [ ]:
lats, lons = data60.variables["latitude"][:], data60.variables["longitude"][:]
no2 = data60.variables['no2'][:]
o3 = data60.variables['go3'][:]
so2 = data60.variables['so2'][:]

fill_dataframes(7, 31, 2019)

In [ ]:
writer = pd.ExcelWriter(template_file_name[:-14] + ' July - Level 60.xlsx', engine='xlsxwriter')

o3_df.to_excel(writer, sheet_name = "o3", index = False)
no2_df.to_excel(writer, sheet_name = "no2", index = False)
so2_df.to_excel(writer, sheet_name = "so2", index = False)

writer.close()

### Level 137 Data

In [ ]:
no2_df = pd.read_excel(template_file_name)
o3_df = pd.read_excel(template_file_name)
so2_df = pd.read_excel(template_file_name)

In [ ]:
lookup_lats, lookup_longs = no2_df["Lat"], no2_df["Long"]

In [ ]:
lats, lons = data137.variables["latitude"][:], data137.variables["longitude"][:]
no2 = data137.variables['no2'][:]
o3 = data137.variables['go3'][:]
so2 = data137.variables['so2'][:]

fill_dataframes(7, 23, 2019) #only reading 23 days since data begins on July 9.

In [ ]:
writer = pd.ExcelWriter(template_file_name[:-14] + ' July - Level 137.xlsx', engine='xlsxwriter')

o3_df.to_excel(writer, sheet_name = "o3", index = False)
no2_df.to_excel(writer, sheet_name = "no2", index = False)
so2_df.to_excel(writer, sheet_name = "so2", index = False)

writer.close()